# PARTE 1: Preprocessing:

- One-hot-encoding
- Undertanding Pipelines (https://www.kaggle.com/thanethunder/pipelines/edit)
- Cross-Validation (https://www.kaggle.com/dansbecker/cross-validation)

In [139]:
import sys

import pandas as pd
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

import pandas_profiling

df = pd.read_csv("bank.csv", sep=";")
type(df)

pandas.core.frame.DataFrame

In [21]:
# pandas_profiling.ProfileReport(df)


# One hot encoding dataset

In [64]:
df1 = pd.get_dummies(df)
df1 = df1.drop(['y_no', "default_no", "housing_no", "loan_no"], axis=1)
df1.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y_yes
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,33,4789,11,220,1,339,4,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,59,0,5,226,1,-1,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0


 CREO LA MATRICE X E IL VETTORE Y 

In [69]:
X = df1.drop("y_yes", axis = 1)
y = df1.y_yes


(4521, 48)
(4521,)


,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,33,4789,11,220,1,339,4,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,59,0,5,226,1,-1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1


Train Test split

In [70]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y)

# Understanding Pipelines
Most scikit-learn objects are either **transformers** or **models.** 

**Transformers** are for pre-processing before modeling.  The Imputer class (for filling in missing values) is an example of a transformer.  Over time, you will learn many more transformers, and you will frequently use multiple transformers sequentially. 

**Models** are used to make predictions. You will usually preprocess your data (with transformers) before putting it in a model.  

You can tell if an object is a transformer or a model by how you apply it.  After fitting a transformer, you apply it with the *transform* command.  After fitting a model, you apply it with the *predict* command. Your pipeline must start with transformer steps and end with a model.  This is what you'd want anyway.

Eventually you will want to apply more transformers and combine them more flexibly.  

Many data scientists hack together models without pipelines, but Pipelines have some important benefits. Those include:
1. **Cleaner Code:** You won't need to keep track of your training (and validation) data at each step of processing.  Accounting for data at each step of processing can get messy.  With a pipeline, you don't need to manually keep track of each step.
2. **Fewer Bugs:** There are fewer opportunities to mis-apply a step or forget a pre-processing step.
3. **Easier to Productionize:** It can be surprisingly hard to transition a model from a prototype to something deployable at scale.  We won't go into the many related concerns here, but pipelines can help.
4. **More Options For Model Testing:** You will see an example in the next tutorial, which covers cross-validation.

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
my_pipeline = make_pipeline(Imputer(), LogisticRegression()) #Transformers, Models

/home/dami/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Avendo creato una pipeline come definita sopra posso procedere all imputazione dei missing e alla stima del modello con una sola riga di codice

In [75]:
my_pipeline.fit(train_X, train_y)
predictions = my_pipeline.predict(test_X)


/home/dami/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


********For comparison, here is the code to do the same thing without pipelines

In [77]:
my_imputer = Imputer()
my_model = RandomForestRegressor()

imputed_train_X = my_imputer.fit_transform(train_X)
imputed_test_X = my_imputer.transform(test_X)
my_model.fit(imputed_train_X, train_y)
predictions = my_model.predict(imputed_test_X)

/home/dami/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/dami/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


# CROSS VALIDATION LOGIT

Imagine you have a dataset with 5000 rows. The train_test_split function has an argument for test_size that you can use to decide how many rows go to the training set and how many go to the test set. The larger the test set, the more reliable your measures of model quality will be. At an extreme, you could imagine having only 1 row of data in the test set. If you compare alternative models, which one makes the best predictions on a single data point will be mostly a matter of luck.

You will typically keep about 20% as a test dataset. But even with 1000 rows in the test set, there's some random chance in determining model scores. A model might do well on one set of 1000 rows, even if it would be inaccurate on a different 1000 rows. The larger the test set, the less randomness (aka "noise") there is in our measure of model quality.

But we can only get a large test set by removing data from our training data, and smaller training datasets mean worse models. In fact, the ideal modeling decisions on a small dataset typically aren't the best modeling decisions on large datasets.

In [96]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
my_pipeline = make_pipeline(Imputer(), LogisticRegressionCV(cv=5, random_state=0))

/home/dami/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [105]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0).fit(X, y)



/home/dami/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/dami/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/dami/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/dami/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/dami/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of

In [142]:
y_pred = clf.predict(X)

clf.predict_proba(X).shape

clf.score(X, y) 

beta = clf.



In [119]:
from sklearn.metrics import confusion_matrix
 
cm = confusion_matrix(y, y_pred)
print(cm)

[[3919   81]
 [ 355  166]]


In [124]:
(3919+ 166)/ (81 +355 +3919+166 )


0.9035611590356116